# Detecção de Fraude em contratos de seguros: Preparação de Dados, Processo e Recursos de Armazenamento

## Background

Este notebook é a segunda parte de uma série de notebooks que demonstrarão como preparar, treinar e implantar um modelo que detecta reclamações automáticas fraudulentas. Neste notebook, estaremos preparando, processando e armazenando recursos usando o SageMaker Feature Store. Você pode optar por executar este notebook sozinho ou em sequência com os outros notebooks listados abaixo. Consulte o [README.md](README.md) para obter mais informações sobre este caso de uso implementado por esta série de notebooks.

1. **[Detecção de Fraude em contratos de seguros: exploração de dados](./0-AutoClaimFraudDetection.ipynb)**
1. [Detecção de Fraude em contratos de seguros: preparação de dados, processo e recursos de armazenamento](./1-data-prep-e2e.ipynb)
1. [Detecção de Fraude em contratos de seguros: treinar, verificar viés, ajustar, registrar linhagem e registrar um modelo](./2-lineage-train-assess-bias-tune-registry-e2e.ipynb)
1. [Detecção de Fraude em contratos de seguros: mitigar, treinar, registrar e implantar modelo imparcial](./3-mitigate-bias-train-model2-registry-e2e.ipynb)


## Conteúdo
1. [Arquitetura para preparação de dados, recursos e armazenamento](#Arquitetura para preparação de dados, recursos de processo e armazenamento)
1. [Introdução: Criando recursos](#Getting-Started:-Creating-Resources)
1. [Conjuntos de dados e tipos de recursos](#Datasets-and-Feature-Types)
1. [SageMaker Feature Store](#SageMaker-Feature-Store)
1. [Criar conjuntos de dados de treinamento e teste](#Create-Train-and-Test-Datasets)

O objetivo deste notebook é realizar a fase de preparação de dados do ciclo de vida de ML. O principal Data Wrangling, ingestão de dados e várias transformações serão feitos por meio da GUI do Data Wrangler do SageMaker Studio.

Neste notebook, pegaremos os arquivos `.flow` que definem as transformações para os dados brutos. e aplique-os usando um trabalho de processamento do SageMaker que aplicará essas transformações aos dados brutos depositados no bucket do S3 como arquivos `.csv`.

## Arquitetura para preparação dos recursos, processo e armazenamento de dados
----
![Data Prep and Store](./images/e2e-1-pipeline-v3b.png)

### Install required and/or update third-party libraries

In [2]:
!python -m pip install -Uq pip
!python -m pip install -q awswrangler==2.2.0 imbalanced-learn==0.7.0 sagemaker==2.41.0 boto3==1.17.70

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 4.13.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
pyathena 2.13.0 requires boto3>=1.21.0, but you have boto3 1.17.70 which is incompatible.
pyathena 2.13.0 requires botocore>=1.24.7, but you have botocore 1.20.112 which is incompatible.
awscli 1.25.27 requires botocore==1.27.27, but you have botocore 1.20.112 which is incompatible.
awscli 1.25.27 requires rsa<4.8,>=3.1.2, but you have rsa 4.8 which is incompatible.
awscli 1.25.27 requires s3transfer<0.7.0,>=0.6.0, but you have s3transfer 0.4.2 which is incompatible.
aiobotocore 2.3.4 requires botocore<1.24.22,>=1.24.21, but you have botocore 1.20.112 which is incompatible.


### Importar bibliotecas

In [3]:
import json
import time
import boto3
import string
import sagemaker
import pandas as pd
import awswrangler as wr

from sagemaker.feature_store.feature_group import FeatureGroup

## Primeiros passos: criando recursos
----
Para executar este notebook com sucesso, você precisará criar alguns recursos da AWS.
Primeiro, um bucket do S3 será criado para armazenar todos os dados deste tutorial.
Depois de criado, você precisará criar uma função do AWS Glue usando o console do IAM e anexar uma política ao bucket do S3 para permitir o acesso do FeatureStore a este notebook. Se você já executou este notebook e está retomando de onde parou, a execução das células abaixo deve retomar os recursos que você já criou sem criar recursos adicionais.

#### Adicione a política FeatureStore à função de execução do Studio

![title](images/iam-policies.png)


1. Em uma guia separada do navegador, vá para a seção IAM do Console AWS
2. Navegue até a seção Funções e selecione a função de execução que você está usando para o usuário do SageMaker Studio
     * Se você não tiver certeza de qual função está usando, execute a célula abaixo para imprimi-la
3. Anexe a política <font color='green'> AmazonSageMakerFeatureStoreAccess </font> a essa função. Uma vez anexado, as alterações entram em vigor imediatamente.

In [4]:
print("SageMaker Role:", sagemaker.get_execution_role().split("/")[-1])

SageMaker Role: AmazonSageMaker-ExecutionRole-20220729T115042


### Selecione a região, boto3 e SageMaker SDK 

In [5]:
# You can change this to a region of your choice
import sagemaker

region = sagemaker.Session().boto_region_name
print("Using AWS Region: {}".format(region))

Using AWS Region: us-east-1


In [6]:
boto3.setup_default_session(region_name=region)

boto_session = boto3.Session(region_name=region)

s3_client = boto3.client("s3", region_name=region)

sagemaker_boto_client = boto_session.client("sagemaker")

sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

In [7]:
"""
Note: if you are not running this notebook from SageMaker Studio or SageMaker Classic Notebooks you will need to instanatiate 
the sagemaker_execution_role_name with an AWS role that has SageMakerFullAccess and SageMakerFeatureStoreFullAccess
"""
sagemaker_execution_role_name = "AmazonSageMaker-ExecutionRole-20210107T234882"
try:
    sagemaker_role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    sagemaker_role = iam.get_role(RoleName=sagemaker_execution_role_name)["Role"]["Arn"]
    print(f"\n instantiating sagemaker_role with supplied role name : {sagemaker_role}")

account_id = boto3.client("sts").get_caller_identity()["Account"]

### Crie um diretório no bucket padrão do SageMaker para este tutorial

In [8]:
if "bucket" not in locals():
    bucket = sagemaker_session.default_bucket()
    prefix = "fraud-detect-demo"
    %store bucket
    %store prefix
    print(f"Creating bucket: {bucket}...")

Stored 'bucket' (str)
Stored 'prefix' (str)
Creating bucket: sagemaker-us-east-1-173965381620...


Se você quiser usar seu próprio bucket do S3 que já existe, remova o comentário e utilize o código de exemplo a seguir.

In [9]:
"""
try:
    s3_client.create_bucket(Bucket=bucket, ACL='private', CreateBucketConfiguration={'LocationConstraint': region})
    print('Create S3 bucket: SUCCESS')
    
except Exception as e:
    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
        print(f'Using existing bucket: {bucket}/{prefix}')
    else:
        raise(e)
"""

"\ntry:\n    s3_client.create_bucket(Bucket=bucket, ACL='private', CreateBucketConfiguration={'LocationConstraint': region})\n    print('Create S3 bucket: SUCCESS')\n    \nexcept Exception as e:\n    if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':\n        print(f'Using existing bucket: {bucket}/{prefix}')\n    else:\n        raise(e)\n"

In [10]:
# ======> Tons of output_paths
traing_job_output_path = f"s3://{bucket}/{prefix}/training_jobs"
bias_report_1_output_path = f"s3://{bucket}/{prefix}/clarify-bias-1"
bias_report_2_output_path = f"s3://{bucket}/{prefix}/clarify-bias-2"
explainability_output_path = f"s3://{bucket}/{prefix}/clarify-explainability"

train_data_uri = f"s3://{bucket}/{prefix}/data/train/train.csv"
test_data_uri = f"s3://{bucket}/{prefix}/data/test/test.csv"

# =======> variables used for parameterizing the notebook run
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

claify_instance_count = 1
clairfy_instance_type = "ml.c5.xlarge"

predictor_instance_count = 1
predictor_instance_type = "ml.c5.xlarge"

### Carregar dados brutos para o S3
Antes que você possa pré-processar os dados brutos com o Data Wrangler, eles devem existir no S3.

In [11]:
s3_client.upload_file(
    Filename="data/claims.csv", Bucket=bucket, Key=f"{prefix}/data/raw/claims.csv"
)
s3_client.upload_file(
    Filename="data/customers.csv", Bucket=bucket, Key=f"{prefix}/data/raw/customers.csv"
)

### Atualizar atributos dentro do arquivo `.flow`
O DataWrangler irá gerar um arquivo .flow. Ele contém uma referência a um bucket do S3 usado durante a Wrangling. Isso pode ser diferente daquele que você tem como padrão neste notebook, por exemplo, se o Wrangling foi feito por outra pessoa, você provavelmente não terá acesso ao seu bucket e agora você precisa apontar para seu próprio bucket do S3 para que você possa realmente carregar o arquivo .flow no Wrangler ou acesse os dados.

Depois de executar a célula abaixo, você pode abrir os arquivos `claims.flow` e `customers.flow` e exportar os dados para o S3 ou pode continuar o guia usando os arquivos `data/claims_preprocessed.csv` e `data/customers_preprocessed.csv fornecidos ` arquivos.

In [12]:
claims_flow_template_file = "claims_flow_template"

with open(claims_flow_template_file, "r") as f:
    variables = {"bucket": bucket, "prefix": prefix}
    template = string.Template(f.read())
    claims_flow = template.substitute(variables)
    claims_flow = json.loads(claims_flow)

with open("claims.flow", "w") as f:
    json.dump(claims_flow, f)

customers_flow_template_file = "customers_flow_template"

with open(customers_flow_template_file, "r") as f:
    variables = {"bucket": bucket, "prefix": prefix}
    template = string.Template(f.read())
    customers_flow = template.substitute(variables)
    customers_flow = json.loads(customers_flow)

with open("customers.flow", "w") as f:
    json.dump(customers_flow, f)

### Carregar dados pré-processados do trabalho do Data Wrangler
Se você executou as tarefas do Data Wrangler de `claims.flow` e `customers.flow`, você pode carregar seus dados pré-processados aqui. Se você não executou o trabalho do Data Wrangler, você ainda pode começar carregando os conjuntos de dados pré-fabricados do diretório `/data` deste exemplo.

## Conjuntos de dados e tipos de recursos
----

In [13]:
claims_dtypes = {
    "policy_id": int,
    "incident_severity": int,
    "num_vehicles_involved": int,
    "num_injuries": int,
    "num_witnesses": int,
    "police_report_available": int,
    "injury_claim": float,
    "vehicle_claim": float,
    "total_claim_amount": float,
    "incident_month": int,
    "incident_day": int,
    "incident_dow": int,
    "incident_hour": int,
    "fraud": int,
    "driver_relationship_self": int,
    "driver_relationship_na": int,
    "driver_relationship_spouse": int,
    "driver_relationship_child": int,
    "driver_relationship_other": int,
    "incident_type_collision": int,
    "incident_type_breakin": int,
    "incident_type_theft": int,
    "collision_type_front": int,
    "collision_type_rear": int,
    "collision_type_side": int,
    "collision_type_na": int,
    "authorities_contacted_police": int,
    "authorities_contacted_none": int,
    "authorities_contacted_fire": int,
    "authorities_contacted_ambulance": int,
    "event_time": float,
}

customers_dtypes = {
    "policy_id": int,
    "customer_age": int,
    "customer_education": int,
    "months_as_customer": int,
    "policy_deductable": int,
    "policy_annual_premium": int,
    "policy_liability": int,
    "auto_year": int,
    "num_claims_past_year": int,
    "num_insurers_past_5_years": int,
    "customer_gender_male": int,
    "customer_gender_female": int,
    "policy_state_ca": int,
    "policy_state_wa": int,
    "policy_state_az": int,
    "policy_state_or": int,
    "policy_state_nv": int,
    "policy_state_id": int,
    "event_time": float,
}

In [14]:
# ======> This is your DataFlow output path if you decide to redo the work in DataFlow on your own
# flow_output_path = <YOUR_PATH_HERE>
claims_flow_path = ""
customers_flow_path = ""

try:
    # this will try to load the exported dataframes from the claims and customers .flow files
    claims_s3_path = f"{flow_output_path}/claims_output"
    customers_s3_path = f"{flow_output_path}/customers_output"

    claims_preprocessed = wr.s3.read_csv(path=claims_s3_path, dataset=True, dtype=claims_dtypes)

    customers_preprocessed = wr.s3.read_csv(
        path=customers_s3_path, dataset=True, dtype=customers_dtypes
    )

except:
    # if the Data Wrangler job was not run, the claims and customers dataframes will be loaded from local copies
    timestamp = pd.to_datetime("now").timestamp()
    print("Unable to load Data Wrangler output. Loading pre-made dataframes...")

    claims_preprocessed = pd.read_csv(
        filepath_or_buffer="data/claims_preprocessed.csv", dtype=claims_dtypes
    )

    # a timestamp column is required by the feature store, so one is added with a current timestamp
    claims_preprocessed["event_time"] = timestamp

    customers_preprocessed = pd.read_csv(
        filepath_or_buffer="data/customers_preprocessed.csv", dtype=customers_dtypes
    )

    customers_preprocessed["event_time"] = timestamp

    print("Complete")

Unable to load Data Wrangler output. Loading pre-made dataframes...
Complete


Agora temos um conjunto de DataFrames do Pandas que contém os dados do cliente e da reclamação, com os tipos de dados corretos. Quando o Dat Wrangler codifica um recurso como um recurso de codificação única, ele usará como padrão os tipos de dados flutuantes para esses recursos resultantes (um recurso --> muitas colunas para a codificação a quente).

<font color ='red'> Nota: </font> a razão para converter explicitamente os tipos de dados para recursos categóricos gerados pelo Data Wrangler é garantir que eles sejam do tipo inteiro para que o Clarify os trate como variáveis categóricas.

## Armazenamento de recursos do SageMaker
----

O Amazon SageMaker Feature Store é um repositório criado especificamente para você armazenar e acessar recursos para que seja muito mais fácil nomeá-los, organizá-los e reutilizá-los entre as equipes. O SageMaker Feature Store fornece um armazenamento unificado de recursos durante o treinamento e inferência em tempo real sem a necessidade de escrever código adicional ou criar processos manuais para manter os recursos consistentes. O SageMaker Feature Store acompanha os metadados dos recursos armazenados (por exemplo, nome do recurso ou número da versão) para que você possa consultar os recursos para os atributos certos em lotes ou em tempo real usando o Amazon Athena, um serviço de consulta interativa. O SageMaker Feature Store também mantém os recursos atualizados, pois à medida que novos dados são gerados durante a inferência, o repositório único é atualizado para que novos recursos estejam sempre disponíveis para os modelos usarem durante o treinamento e a inferência.

Um repositório de recursos consiste em um componente offline armazenado no S3 e um componente online armazenado em um banco de dados de baixa latência. O banco de dados online é opcional, mas muito útil se você precisar que recursos suplementares estejam disponíveis na inferência. Nesta seção, criaremos grupos de recursos para nossos conjuntos de dados de Reclamações e Clientes. Depois de inserir as declarações e os dados do cliente em seus respectivos grupos de recursos, você precisa consultar a armazenamento offline com o Athena para criar o conjunto de dados de treinamento.

Você pode consultar o [Guia do desenvolvedor do SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store.html) para obter mais informações sobre o SageMaker Feature Store.

In [15]:
featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = sagemaker.Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_boto_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

### Configure os grupos de recursos
O tipo de dados para cada recurso é definido passando um dataframe e inferindo o tipo de dados apropriado. Os tipos de dados de recursos também podem ser definidos por meio de uma variável de configuração, mas terão que corresponder ao tipo de dados Python correspondente no dataframe do Pandas quando forem ingeridos no Grupo de recursos.

In [16]:
claims_fg_name = f"{prefix}-claims"
customers_fg_name = f"{prefix}-customers"
%store claims_fg_name
%store customers_fg_name

claims_feature_group = FeatureGroup(name=claims_fg_name, sagemaker_session=feature_store_session)

customers_feature_group = FeatureGroup(
    name=customers_fg_name, sagemaker_session=feature_store_session
)

claims_feature_group.load_feature_definitions(data_frame=claims_preprocessed)
customers_feature_group.load_feature_definitions(data_frame=customers_preprocessed);

Stored 'claims_fg_name' (str)
Stored 'customers_fg_name' (str)


### Crie os grupos de recursos
Você deve informar ao Feature Group quais colunas no dataframe correspondem ao identificador de registro necessário e aos recursos de tempo do evento.

In [17]:
print(f"{customers_fg_name} -- {claims_fg_name} are the feature group names in use")

fraud-detect-demo-customers -- fraud-detect-demo-claims are the feature group names in use


In [18]:
record_identifier_feature_name = "policy_id"
event_time_feature_name = "event_time"

try:
    print(f"\n Using s3://{bucket}/{prefix}")
    claims_feature_group.create(
        s3_uri=f"s3://{bucket}/{prefix}",
        record_identifier_name=record_identifier_feature_name,
        event_time_feature_name=event_time_feature_name,
        role_arn=sagemaker_role,
        enable_online_store=True,
    )
    print(f'Create "claims" feature group: SUCCESS')
except Exception as e:
    code = e.response.get("Error").get("Code")
    if code == "ResourceInUse":
        print(f"Using existing feature group: {claims_fg_name}")
    else:
        raise (e)

try:
    customers_feature_group.create(
        s3_uri=f"s3://{bucket}/{prefix}",
        record_identifier_name=record_identifier_feature_name,
        event_time_feature_name=event_time_feature_name,
        role_arn=sagemaker_role,
        enable_online_store=True,
    )
    print(f'Create "customers" feature group: SUCCESS')
except Exception as e:
    code = e.response.get("Error").get("Code")
    if code == "ResourceInUse":
        print(f"Using existing feature group: {customers_fg_name}")
    else:
        raise (e)


 Using s3://sagemaker-us-east-1-173965381620/fraud-detect-demo
Create "claims" feature group: SUCCESS
Create "customers" feature group: SUCCESS


### Aguarde até que a criação do grupo de recursos seja totalmente concluída

In [19]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")


wait_for_feature_group_creation_complete(feature_group=claims_feature_group)
wait_for_feature_group_creation_complete(feature_group=customers_feature_group)

Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup fraud-detect-demo-claims successfully created.
Waiting for Feature Group Creation
FeatureGroup fraud-detect-demo-customers successfully created.


### Ingerir registros nos grupos de recursos
Após a criação dos grupos de recursos, podemos colocar dados em cada armazenamento usando a API PutRecord. Essa API pode lidar com TPS alto e foi projetada para ser chamada por diferentes fluxos. Os dados de todas essas solicitações Put são armazenados em buffer e gravados em s3 em partes. Os arquivos serão gravados no armazenamento offline alguns minutos após a ingestão.

In [20]:
if "claims_table" in locals():
    print(
        "You may have already ingested the data into your Feature Groups. If you'd like to do this again, you can run the ingest methods outside of the 'if/else' statement."
    )

else:
    claims_feature_group.ingest(data_frame=claims_preprocessed, max_workers=3, wait=True)

    customers_feature_group.ingest(data_frame=customers_preprocessed, max_workers=3, wait=True);

### Aguarde até que os dados de armazenamento offline fiquem disponíveis
Isso geralmente leva de 5 a 8 minutos

In [21]:
if "claims_table" not in locals():
    claims_table = claims_feature_group.describe()["OfflineStoreConfig"]["DataCatalogConfig"][
        "TableName"
    ]
if "customers_table" not in locals():
    customers_table = customers_feature_group.describe()["OfflineStoreConfig"]["DataCatalogConfig"][
        "TableName"
    ]

claims_feature_group_s3_prefix = (
    f"{prefix}/{account_id}/sagemaker/{region}/offline-store/{claims_table}/data"
)
customers_feature_group_s3_prefix = (
    f"{prefix}/{account_id}/sagemaker/{region}/offline-store/{customers_table}/data"
)

print(claims_feature_group_s3_prefix)

offline_store_contents = None
while offline_store_contents is None:
    objects_in_bucket = s3_client.list_objects(
        Bucket=bucket, Prefix=customers_feature_group_s3_prefix
    )
    if "Contents" in objects_in_bucket and len(objects_in_bucket["Contents"]) > 1:
        offline_store_contents = objects_in_bucket["Contents"]
    else:
        print("Waiting for data in offline store...")
        time.sleep(60)

print("\nData available.")

fraud-detect-demo/173965381620/sagemaker/us-east-1/offline-store/fraud-detect-demo-claims-1661685235/data
Waiting for data in offline store...
Waiting for data in offline store...
Waiting for data in offline store...
Waiting for data in offline store...
Waiting for data in offline store...

Data available.


In [22]:
claims_feature_group.describe()["OfflineStoreConfig"][
    "DataCatalogConfig"
], customers_feature_group.describe()["OfflineStoreConfig"]["DataCatalogConfig"]

({'TableName': 'fraud-detect-demo-claims-1661685235',
  'Catalog': 'AwsDataCatalog',
  'Database': 'sagemaker_featurestore'},
 {'TableName': 'fraud-detect-demo-customers-1661685237',
  'Catalog': 'AwsDataCatalog',
  'Database': 'sagemaker_featurestore'})

## Crie conjuntos de dados de treinamento e teste
----

Assim que os dados estiverem disponíveis no armazenamento offline, eles serão automaticamente catalogados e carregados em uma tabela do Athena (isso é feito por padrão, mas pode ser desativado). Para construir nossos conjuntos de dados de treinamento e teste, você enviará uma consulta SQL para unir as tabelas Claims e Customers criadas no Athena.

In [23]:
claims_query = claims_feature_group.athena_query()
customers_query = customers_feature_group.athena_query()

claims_table = claims_query.table_name
customers_table = customers_query.table_name
database_name = customers_query.database
%store claims_table
%store customers_table
%store database_name

feature_columns = list(set(claims_preprocessed.columns) ^ set(customers_preprocessed.columns))
feature_columns_string = ", ".join(f'"{c}"' for c in feature_columns)
feature_columns_string = f'"{claims_table}".policy_id as policy_id, ' + feature_columns_string

query_string = f"""
SELECT DISTINCT {feature_columns_string}
FROM "{claims_table}" LEFT JOIN "{customers_table}" 
ON "{claims_table}".policy_id = "{customers_table}".policy_id
"""

Stored 'claims_table' (str)
Stored 'customers_table' (str)
Stored 'database_name' (str)


In [24]:
claims_query.run(query_string=query_string, output_location=f"s3://{bucket}/{prefix}/query_results")
claims_query.wait()
dataset = claims_query.as_dataframe()

In [25]:
dataset.to_csv("./data/claims_customer.csv")

In [26]:
col_order = ["fraud"] + list(dataset.drop(["fraud", "policy_id"], axis=1).columns)

train = dataset.sample(frac=0.80, random_state=0)[col_order]
test = dataset.drop(train.index)[col_order]

### Gravar treino, testar dados para S3

In [27]:
train.to_csv("data/train.csv", index=False)
test.to_csv("data/test.csv", index=False)
dataset.to_csv("data/dataset.csv", index=True)

In [28]:
train.head(5)

,fraud,collision_type_rear,customer_age,vehicle_claim,incident_severity,authorities_contacted_ambulance,policy_liability,total_claim_amount,auto_year,num_injuries,...,incident_day,injury_claim,policy_deductable,authorities_contacted_fire,police_report_available,customer_gender_male,policy_state_az,policy_state_nv,authorities_contacted_none,policy_state_id
398,1,1,57,6000.0,0,0,0,6000.0,2017,0,...,2,0.0,750,0,0,1,0,0,1,0
3833,0,0,19,9500.0,0,0,1,9500.0,2016,0,...,5,0.0,750,0,1,1,0,0,0,0
4836,0,0,39,10000.0,0,0,0,10000.0,2019,0,...,8,0.0,750,0,0,1,0,0,0,0
4572,0,1,49,32000.0,2,0,2,49500.0,2012,1,...,9,17500.0,750,0,1,0,0,0,0,1
636,0,0,57,8000.0,1,0,2,38000.0,2005,1,...,29,30000.0,750,0,1,0,0,0,0,0


In [29]:
test.head(5)

,fraud,collision_type_rear,customer_age,vehicle_claim,incident_severity,authorities_contacted_ambulance,policy_liability,total_claim_amount,auto_year,num_injuries,...,incident_day,injury_claim,policy_deductable,authorities_contacted_fire,police_report_available,customer_gender_male,policy_state_az,policy_state_nv,authorities_contacted_none,policy_state_id
0,0,0,30,15500.0,0,0,1,15500.0,2019,0,...,31,0.0,750,0,0,1,0,0,0,0
7,0,0,38,7500.0,0,0,1,7500.0,2012,0,...,23,0.0,750,0,0,1,0,0,0,0
21,0,1,53,28000.0,2,0,2,28000.0,2016,0,...,8,0.0,750,0,1,1,0,0,0,0
24,0,0,39,15500.0,1,0,2,15500.0,2019,0,...,15,0.0,750,0,0,0,0,0,0,0
25,0,0,41,9500.0,0,0,1,9500.0,2020,0,...,30,0.0,750,0,0,1,0,0,0,0
